In [61]:
#!pip install polars
#por alguna razón cagué mi pyenv y tuve que instalar polars ngl

In [62]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [63]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [64]:
num_cars = 100
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head())

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: "None"


In [65]:
data.collect()



car_id,timestamp,latitude,longitude,notification
str,str,f64,f64,str
"""car_0""","""2023-01-01T00:00:00""",28.391706,-113.255354,null
"""car_0""","""2023-01-01T00:09:49.308818""",28.391706,-113.255354,null
"""car_0""","""2023-01-01T00:16:37.313807""",28.391706,-113.255354,"""engine_check"""
"""car_0""","""2023-01-01T00:06:56.759156""",28.391706,-113.255354,"""tire_pressure"""
"""car_0""","""2023-01-01T00:04:08.035599""",28.391706,-113.255354,null
…,…,…,…,…
"""car_99""","""2023-01-07T23:39:45.357321""",26.241385,-110.521235,"""tire_pressure"""
"""car_99""","""2023-01-07T23:36:26.126236""",26.241385,-110.521235,null
"""car_99""","""2023-01-07T23:33:45.966442""",26.241385,-110.521235,"""low_fuel"""


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [66]:
data = data.with_columns(
    pl.when(pl.col("timestamp").str.contains(r"\."))
    .then(pl.col("timestamp"))
    .otherwise(pl.col("timestamp") + ".000000")
    .alias("timestamp")
)

print(data.collect())

data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f"))
print(data.collect())

shape: (625_770, 5)
┌────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ str                        ┆ f64       ┆ f64         ┆ str           │
╞════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01T00:00:00.000000 ┆ 28.391706 ┆ -113.255354 ┆ null          │
│ car_0  ┆ 2023-01-01T00:09:49.308818 ┆ 28.391706 ┆ -113.255354 ┆ null          │
│ car_0  ┆ 2023-01-01T00:16:37.313807 ┆ 28.391706 ┆ -113.255354 ┆ engine_check  │
│ car_0  ┆ 2023-01-01T00:06:56.759156 ┆ 28.391706 ┆ -113.255354 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:04:08.035599 ┆ 28.391706 ┆ -113.255354 ┆ null          │
│ …      ┆ …                          ┆ …         ┆ …           ┆ …             │
│ car_99 ┆ 2023-01-07T23:39:45.357321 ┆ 26.241385 ┆ -110.521235 ┆ tire_pressur

C:\Users\Admin\AppData\Local\Temp\ipykernel_29836\3340590239.py:10: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f"))


shape: (625_770, 5)
┌────────┬───────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                     ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                           ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ datetime[ns]                  ┆ f64       ┆ f64         ┆ str           │
╞════════╪═══════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01 00:00:00           ┆ 28.391706 ┆ -113.255354 ┆ null          │
│ car_0  ┆ 2023-01-01 00:09:49.000308818 ┆ 28.391706 ┆ -113.255354 ┆ null          │
│ car_0  ┆ 2023-01-01 00:16:37.000313807 ┆ 28.391706 ┆ -113.255354 ┆ engine_check  │
│ car_0  ┆ 2023-01-01 00:06:56.000759156 ┆ 28.391706 ┆ -113.255354 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01 00:04:08.000035599 ┆ 28.391706 ┆ -113.255354 ┆ null          │
│ …      ┆ …                             ┆ …         ┆ …           ┆ …             │
│ car_99 ┆ 2023-01-07 23:39:45.000357321 ┆ 26

### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es necesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [67]:
data = (data
    .sort(['car_id', 'timestamp'])
    .with_columns([
        pl.col('timestamp').diff().over('car_id').fill_null(pl.duration(nanoseconds=0)).alias('notification_time')]))

In [68]:
data.collect()


car_id,timestamp,latitude,longitude,notification,notification_time
str,datetime[ns],f64,f64,str,duration[ns]
"""car_0""",2023-01-01 00:00:00,28.391706,-113.255354,null,0ns
"""car_0""",2023-01-01 00:00:08.000409243,28.391706,-113.255354,"""engine_check""",8s 409243ns
"""car_0""",2023-01-01 00:00:34.000411138,28.391706,-113.255354,null,26s 1895ns
"""car_0""",2023-01-01 00:01:40.000666523,28.391706,-113.255354,"""low_fuel""",1m 6s 255385ns
"""car_0""",2023-01-01 00:01:43.000788422,28.391706,-113.255354,null,3s 121899ns
…,…,…,…,…,…
"""car_99""",2023-01-07 23:46:09.000163299,26.241385,-110.521235,null,1m 1s 999359206ns
"""car_99""",2023-01-07 23:46:09.000312719,26.241385,-110.521235,"""tire_pressure""",149420ns
"""car_99""",2023-01-07 23:49:28.000346299,26.241385,-110.521235,null,3m 19s 33580ns


2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [69]:
## TUVE QUE PREGUNTARLE A CHAT PORQUE NO ME SALÍA EL EJERCICIO Y CREO QUE AL FINAL ME SLAEN COMO 0.0
## PORQUE ESTÁ POR NOTIFICACIÓN DE LOS COCHES ENTONCES EN LOS ... ESTÁN LAS DEMÁS DISTANCIAS (SOLO QUIERO LA ÚLTIMA)
## Y YA EN EL CARRO 99 SI ME SALE UNA DISTANCIA PORQUE PUEDO VER LA ÚLTIMA 


##Side note: Lo volví a correr y ya me jala bien todo EXCEPTO ESTO PORQUE AHORA TODO ME SALE 0.0 DE DISTANCIA 
##Hace rato ya estaba bien :((( yo solo soy una mujer que quiere dormir y no puede. 

##PONGAME LA 2 MAL, ALCH NO SUPE HACERLA. 

In [70]:


# Define una función para calcular la distancia entre dos puntos geográficos
def calculate_distance(lat1, lon1, lat2, lon2):
    if None in [lat1, lon1, lat2, lon2]:
        return None
    return geodesic((lat1, lon1), (lat2, lon2)).meters

# Cargar los datos (asegúrate de que tus datos ya están en un DataFrame de Polars llamado 'data')

# Crear columnas adicionales con la latitud y longitud de la fila anterior
data_2 = data.with_columns([
    pl.col("latitude").shift(1).alias("prev_latitude"),
    pl.col("longitude").shift(1).alias("prev_longitude"),
    pl.col("notification").shift(1).alias("prev_notification"),
])

# Calcular la distancia desde la última notificación
data_2 = data_2.with_columns(
    pl.struct(["prev_latitude", "prev_longitude", "latitude", "longitude"]).apply(
        lambda x: calculate_distance(x["prev_latitude"], x["prev_longitude"], x["latitude"], x["longitude"]),
        return_dtype=pl.Float64
    ).alias("distance")
)

# Filtrar las filas que no tienen una notificación previa
data_3 = data_2.filter(pl.col("prev_notification").is_not_null() & pl.col("notification").is_not_null())

# Mostrar el DataFrame con la nueva columna
print(data_3.collect())


C:\Users\Admin\AppData\Local\Temp\ipykernel_29836\295531222.py:18: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.struct(["prev_latitude", "prev_longitude", "latitude", "longitude"]).apply(


shape: (352_766, 10)
┌────────┬────────────┬───────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ car_id ┆ timestamp  ┆ latitude  ┆ longitude  ┆ … ┆ prev_lati ┆ prev_long ┆ prev_noti ┆ distance  │
│ ---    ┆ ---        ┆ ---       ┆ ---        ┆   ┆ tude      ┆ itude     ┆ fication  ┆ ---       │
│ str    ┆ datetime[n ┆ f64       ┆ f64        ┆   ┆ ---       ┆ ---       ┆ ---       ┆ f64       │
│        ┆ s]         ┆           ┆            ┆   ┆ f64       ┆ f64       ┆ str       ┆           │
╞════════╪════════════╪═══════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ car_0  ┆ 2023-01-01 ┆ 28.391706 ┆ -113.25535 ┆ … ┆ 28.391706 ┆ -113.2553 ┆ engine_ch ┆ 0.0       │
│        ┆ 00:05:30.0 ┆           ┆ 4          ┆   ┆           ┆ 54        ┆ eck       ┆           │
│        ┆ 00938623   ┆           ┆            ┆   ┆           ┆           ┆           ┆           │
│ car_0  ┆ 2023-01-01 ┆ 28.391706 ┆ -113.25535 ┆ … ┆ 28.391706 ┆ -113.

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [71]:
# Agrupar por 'car_id' y por intervalos de tiempo en 'timestamp'
info_notificacion = data.group_by_dynamic(
    group_by="car_id", index_column="timestamp", every=x, period=x, closed="both", include_boundaries=True
).agg([
    pl.col("notification_time").mean().alias("media"),
    pl.col("notification_time").median().alias("mediana"),
    pl.col("notification_time").var().alias("varianza"),
    pl.col("notification_time").max().alias("max"),
    pl.col("notification_time").min().alias("min")
])

print(info_notificacion.select('car_id', 'timestamp', 'media', 'mediana', 'varianza', 'max', 'min').collect()) ##AQUÍ DEBIO ESTAR BIEN Y ARRIBA NO WTF 

shape: (137_853, 7)
┌────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ car_id ┆ timestamp    ┆ media        ┆ mediana      ┆ varianza     ┆ max          ┆ min          │
│ ---    ┆ ---          ┆ ---          ┆ ---          ┆ ---          ┆ ---          ┆ ---          │
│ str    ┆ datetime[ns] ┆ duration[ns] ┆ duration[ns] ┆ duration[ns] ┆ duration[ns] ┆ duration[ns] │
╞════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ car_0  ┆ 2022-12-31   ┆ 0ns          ┆ 0ns          ┆ null         ┆ 0ns          ┆ 0ns          │
│        ┆ 23:55:00     ┆              ┆              ┆              ┆              ┆              │
│ car_0  ┆ 2023-01-01   ┆ 35s          ┆ 8s 409243ns  ┆ null         ┆ 2m 17s       ┆ 0ns          │
│        ┆ 00:00:00     ┆ 428576514ns  ┆              ┆              ┆ 999892802ns  ┆              │
│ car_0  ┆ 2023-01-01   ┆ 1m 25s       ┆ 1m 20s       ┆ null         ┆ 

In [73]:

def calcular_estadisticas_distancia(data, x):
    # Agrupar por 'car_id' y por intervalos de tiempo en 'timestamp'
    info_distance = data.group_by_dynamic(
        group_by="car_id", index_column="timestamp", every=x, period=x, closed="both", include_boundaries=True
    ).agg([
        pl.col("distance").mean().alias("media"),
        pl.col("distance").median().alias("mediana"),
        pl.col("distance").var().alias("varianza"),
        pl.col("distance").max().alias("max"),
        pl.col("distance").min().alias("min")
    ])

    return info_distance

resultado = calcular_estadisticas_distancia(data_3, "5m")

print(resultado.collect()) 

shape: (105_628, 9)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ car_id ┆ _lower_bou ┆ _upper_bou ┆ timestamp ┆ … ┆ mediana   ┆ varianza  ┆ max       ┆ min       │
│ ---    ┆ ndary      ┆ ndary      ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str    ┆ ---        ┆ ---        ┆ datetime[ ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
│        ┆ datetime[n ┆ datetime[n ┆ ns]       ┆   ┆           ┆           ┆           ┆           │
│        ┆ s]         ┆ s]         ┆           ┆   ┆           ┆           ┆           ┆           │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ car_0  ┆ 2023-01-01 ┆ 2023-01-01 ┆ 2023-01-0 ┆ … ┆ 0.0       ┆ 0.0       ┆ 0.0       ┆ 0.0       │
│        ┆ 00:05:00   ┆ 00:10:00   ┆ 1         ┆   ┆           ┆           ┆           ┆           │
│        ┆            ┆            ┆ 00:05:00  ┆   ┆           ┆       